In [ ]:
import os.path

import numpy as np
import pandas as pd
from scipy.misc import central_diff_weights
from sklearn.gaussian_process import GaussianProcessRegressor

from sklearn.gaussian_process.kernels import RBF, ConstantKernel, Matern

import datetime

from scipy.stats import norm

import matplotlib.pyplot as plt

from scipy.interpolate import UnivariateSpline

np.random.seed(123)

In [ ]:
def find_vol(target_value, call_put, S, K, T, r):
    """http://www.codeandfinance.com/finding-implied-vol.html"""
    MAX_ITERATIONS = 100
    PRECISION = 1.0e-5

    sigma = 0.5
    for i in range(0, MAX_ITERATIONS):
        price = bs_price(call_put, S, K, T, r, sigma)
        vega = bs_vega(call_put, S, K, T, r, sigma)

        price = price
        diff = target_value - price  # our root

        if (abs(diff) < PRECISION):
            return sigma
        sigma = sigma + diff/vega # f(x) / f'(x)

    # value wasn't found, return best guess so far
    return sigma

def bs_price(cp_flag,S,K,T,r,v,q=0.0):
    n = norm.pdf
    N = norm.cdf
    d1 = (np.log(S/K)+(r+v*v/2.)*T)/(v*np.sqrt(T))
    d2 = d1-v*np.sqrt(T)
    if cp_flag == 'c':
        price = S*np.exp(-q*T)*N(d1)-K*np.exp(-r*T)*N(d2)
    else:
        price = K*np.exp(-r*T)*N(-d2)-S*np.exp(-q*T)*N(-d1)
    return price


def bs_vega(cp_flag,S,K,T,r,v,q=0.0):
    n = norm.pdf
    N = norm.cdf
    d1 = (np.log(S/K)+(r+v*v/2.)*T)/(v*np.sqrt(T))
    return S * np.sqrt(T)*n(d1)

In [ ]:
"""test example
V_market = 17.5
K = 585
T = (datetime.date(2014,10,18) - datetime.date(2014,9,8)).days / 365.
S = 586.08
r = 0.0002
cp = 'c' # call option

implied_vol = find_vol(V_market, cp, S, K, T, r)

print('Implied vol: %.2f%%' % (implied_vol * 100))
print('Market price = %.2f' % V_market)
print('Model price = %.2f' % bs_price(cp, S, K, T, r, implied_vol))
# Implied vol: 21.92%
# Market price = 17.50
# Model price = 17.50
"""

In [ ]:
# TODO must set these before run
data_set = 'foo'
spot_price = 40
price_grid = np.linspace(20, 60, 100)

data_date = pd.datetime(2019, 9, 30)
risk_free_rate = 1.8 / 100
plot_samples = 10
mc_samples = 2000
spline_pts = 5

path = os.path.join('./data', data_set)

In [ ]:
df_20191101 = pd.read_csv(os.path.join(path, '20191101.tsv'), delimiter='\t', header=0, index_col=False, na_values=['-'], comment='#')
df_20191101['expiry'] = pd.datetime(2019, 11, 1)

df_20200117 = pd.read_csv(os.path.join(path, '20200117.tsv'), delimiter='\t', header=0, index_col=False, na_values=['-'], comment='#')
df_20200117['expiry'] = pd.datetime(2020, 1, 17)

df_20210115 = pd.read_csv(os.path.join(path, '20210115.tsv'), delimiter='\t', header=0, index_col=False, na_values=['-'], comment='#')
df_20210115['expiry'] = pd.datetime(2021, 1, 15)

In [ ]:
calls_df = pd.concat([df_20191101, df_20200117, df_20210115], axis=0, ignore_index=True)

calls_df['TTE_days'] = (calls_df['expiry'] - data_date).apply(lambda x: int(x.days))
calls_df['TTE_years'] = calls_df['TTE_days'] / 365.0
calls_df['Mid'] = 0.5 * (calls_df['Bid'] + calls_df['Ask'])
calls_df['Spread'] = calls_df['Ask'] - calls_df['Bid']
calls_df['Implied Volatility'] = calls_df['Implied Volatility'].apply(lambda vv: float(vv[:-1]) / 100)

calls_df['IV_ask'] = calls_df.apply(lambda row: find_vol(row['Ask'], 'c', spot_price, row['Strike'], row['TTE_years'], risk_free_rate), axis=1)
calls_df['IV_bid'] = calls_df.apply(lambda row: find_vol(row['Bid'], 'c', spot_price, row['Strike'], row['TTE_years'], risk_free_rate), axis=1)
calls_df['IV_mid'] = 0.5 * (calls_df['IV_bid'] + calls_df['IV_ask'])
calls_df['IV_spread'] = (calls_df['IV_ask'] - calls_df['IV_bid']).abs()

print(calls_df)

In [ ]:
X = calls_df[['TTE_years', 'Strike']].values
y = calls_df['IV_mid'].values  # TODO try doing this in log scale
alpha_var = (calls_df['IV_spread']**2) / 12.0  # moment match to uniform

# Remove nans where BS fails to converge
missing = np.isnan(y)
X = X[~missing, :]
y = y[~missing]
alpha_var = alpha_var[~missing]

In [ ]:
base_kernel = Matern(
    nu=5.0 / 2.0,
    length_scale=[10, 10],
    length_scale_bounds=(.1, 1000),
)

k1 = ConstantKernel(
    constant_value=1.0, constant_value_bounds=(0.01, 10000.0)
)

kernel = k1 * base_kernel

gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10, normalize_y=False, alpha=alpha_var, random_state=0)
gp.fit(X, y)
gp.kernel_.get_params()

In [ ]:
for tte in np.unique(calls_df['TTE_years'].values):
    X_test = np.zeros((len(price_grid), 2))
    X_test[:, 0] = tte
    X_test[:, 1] = price_grid

    idx = (calls_df['TTE_years'] == tte)

    mu, cov = gp.predict(X_test, return_cov=True)
    plt.figure()
    plt.plot(price_grid, mu, 'k-')
    plt.plot(price_grid, mu + 2 * np.sqrt(np.diag(cov)), 'r-')
    plt.plot(price_grid, mu - 2 * np.sqrt(np.diag(cov)), 'r-')
    plt.plot(calls_df['Strike'][idx].values, calls_df['IV_ask'][idx].values, '.-')
    plt.plot(calls_df['Strike'][idx].values, calls_df['IV_bid'][idx].values, '.-')
    plt.grid()
    plt.title(tte)

In [ ]:
for tte in np.unique(calls_df['TTE_years'].values):
    X_test = np.zeros((len(price_grid), 2))
    X_test[:, 0] = tte
    X_test[:, 1] = price_grid

    idx = (calls_df['TTE_years'] == tte)

    mu, cov = gp.predict(X_test, return_cov=True)

    IV_vecs = np.random.multivariate_normal(mu, cov, size=plot_samples)
    
    plt.figure()
    plt.plot(price_grid, mu, 'k-')
    plt.plot(price_grid, mu + 2 * np.sqrt(np.diag(cov)), 'r-')
    plt.plot(price_grid, mu - 2 * np.sqrt(np.diag(cov)), 'r-')
    plt.plot(price_grid, IV_vecs.T)
    plt.plot(calls_df['Strike'][idx].values, calls_df['IV_ask'][idx].values, '.-')
    plt.plot(calls_df['Strike'][idx].values, calls_df['IV_bid'][idx].values, '.-')
    plt.grid()
    plt.title(tte)

In [ ]:
for tte in np.unique(calls_df['TTE_years'].values):
    X_test = np.zeros((len(price_grid), 2))
    X_test[:, 0] = tte
    X_test[:, 1] = price_grid

    idx = (calls_df['TTE_years'] == tte)

    mu, cov = gp.predict(X_test, return_cov=True)

    call_price = np.zeros((len(price_grid),))
    for ii in range(len(price_grid)):
        call_price[ii] = bs_price('c', spot_price, price_grid[ii], tte, risk_free_rate, mu[ii])

    plt.figure()
    plt.plot(price_grid, call_price, 'k')
    plt.plot(calls_df['Strike'][idx].values, calls_df['Mid'][idx].values, '.-')
    plt.plot(calls_df['Strike'][idx].values, calls_df['Mid'][idx].values, '.-')
    plt.grid()
    plt.title(tte)

In [ ]:
for tte in np.unique(calls_df['TTE_years'].values):
    X_test = np.zeros((len(price_grid), 2))
    X_test[:, 0] = tte
    X_test[:, 1] = price_grid

    idx = (calls_df['TTE_years'] == tte)

    mu, cov = gp.predict(X_test, return_cov=True)

    IV_vecs = np.random.multivariate_normal(mu, cov, size=plot_samples)

    call_price = np.zeros((plot_samples, len(price_grid)))
    for nn in range(plot_samples):
        for ii in range(len(price_grid)):
            call_price[nn, ii] = bs_price('c', spot_price, price_grid[ii], tte, risk_free_rate, IV_vecs[nn, ii])

    plt.figure()
    plt.plot(price_grid, call_price.T)
    plt.plot(calls_df['Strike'][idx].values, calls_df['Mid'][idx].values, '.-')
    plt.plot(calls_df['Strike'][idx].values, calls_df['Mid'][idx].values, '.-')
    plt.grid()
    plt.title(tte)

In [ ]:
t_hold = 50

for tte in np.unique(calls_df['TTE_years'].values):
    X_test = np.zeros((len(price_grid), 2))
    X_test[:, 0] = tte
    X_test[:, 1] = price_grid

    idx = (calls_df['TTE_years'] == tte)

    mu, cov = gp.predict(X_test, return_cov=True)

    call_price = np.zeros((len(price_grid),))
    call_price2 = np.zeros((len(price_grid),))
    for ii in range(len(price_grid)):
        call_price[ii] = bs_price('c', spot_price, price_grid[ii], tte, risk_free_rate, mu[ii])
    spl = UnivariateSpline(price_grid, call_price, k=spline_pts, s=0)
    call_price2 = spl(price_grid, nu=2)

    call_price2 = np.maximum(call_price2, 0)
    call_price2 = call_price2 / np.sum(call_price2)
    
    plt.figure()
    plt.plot(price_grid, call_price2)
    plt.grid()
    plt.title(tte)

    mean_p = np.sum(price_grid * call_price2)

    idx = price_grid <= t_hold
    p_thold = np.sum(call_price2[idx])
    
    print(mean_p, p_thold)
    plt.plot([mean_p, mean_p], [0, max(call_price2)])

In [ ]:
for tte in np.unique(calls_df['TTE_years'].values):
    X_test = np.zeros((len(price_grid), 2))
    X_test[:, 0] = tte
    X_test[:, 1] = price_grid

    idx = (calls_df['TTE_years'] == tte)

    mu, cov = gp.predict(X_test, return_cov=True)

    IV_vecs = np.random.multivariate_normal(mu, cov, size=mc_samples)

    call_price = np.zeros((mc_samples, len(price_grid)))
    call_price2 = np.zeros((mc_samples, len(price_grid)))
    for nn in range(mc_samples):
        for ii in range(len(price_grid)):
            call_price[nn, ii] = bs_price('c', spot_price, price_grid[ii], tte, risk_free_rate, IV_vecs[nn, ii])
        spl = UnivariateSpline(price_grid, call_price[nn, :], k=spline_pts, s=0)
        call_price2[nn, :] = spl(price_grid, nu=2)

    plt.figure()
    plt.plot(price_grid, call_price2[:plot_samples,:].T)
    plt.plot(price_grid, np.mean(call_price2, axis=0), 'k', lw=2)
    plt.grid()
    plt.title(tte)
    
    call_price2 = np.mean(call_price2, axis=0)
    call_price2 = np.maximum(call_price2, 0)
    call_price2 = call_price2 / np.sum(call_price2)
    
    mean_p = np.sum(price_grid * call_price2)
    print(mean_p)
    plt.plot([mean_p, mean_p], [0, max(call_price2)])